In [ ]:
from datasets import load_dataset

dataset = load_dataset("jacob-hugging-face/job-descriptions")


In [ ]:
print(dataset['train'].to_dict())

Convert the dictionry data into json format

In [ ]:
# Transforming the data into horizontal format
import json
horizontal_data = []
count=0
for item in dataset["train"]:

    if(count>=5):
        break
    horizontal_item = {
        "company_name": item["company_name"],
        "job_description": item["job_description"],
        "position_title": item["position_title"],
        "description_length": item["description_length"],
        "model_response": item["model_response"]
    }
    horizontal_data.append(horizontal_item)
    count+=1

# Saving the horizontal data to a JSON file
with open("cv_data.json", "w") as file:
    json.dump(horizontal_data, file)

Perform data extraction from the pdf.

In [ ]:
import os
import pdfplumber
import json

# Function to extract category, skills, and education from a PDF
def extract_details(pdf_path):
    try:
        with pdfplumber.open(pdf_path) as pdf:
            # Initialize variables to store extracted details
            category = None
            skills = []
            education = []

            # Iterate through pages in the PDF
            for page in pdf.pages:
                text = page.extract_text()

                # Search for patterns in the extracted text
                if "Category" in text:
                    category = text.split("Category")[1].strip()
                if "Skills" in text:
                    skills = [skill.strip() for skill in text.split("Skills")[1].split(",")]
                if "Education" in text:
                    education = [edu.strip() for edu in text.split("Education")[1].split(";")]

            return {
                'PDFFilename': os.path.basename(pdf_path), # Include the PDF filename
                'Category': category,
                'Skills': skills,
                'Education': education,
            }
    except Exception as e:
        print(f"Error extracting details from {pdf_path}: {str(e)}")
        return None

# Directory containing PDF CVs
pdf_directory = '/media/asifr/work/cv matches/archive/data/data/ENGINEERING'

# Create a list to store extracted details
all_details = []

# Iterate through PDF files and extract details
for filename in os.listdir(pdf_directory):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_directory, filename)
        details = extract_details(pdf_path)
        if details:
            print(f"Details extracted from {filename}:\n{details}\n")
            all_details.append(details)

# Save the extracted details in a JSON file
output_file = 'extracted/extracted_details.json'
with open(output_file, 'w') as json_file:
    json.dump(all_details, json_file, indent=4)

print(f"Extracted details saved to {output_file}")


Tokenize and preprocessing

In [86]:
import os
import json
import torch
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.metrics.pairwise import cosine_similarity

# Load DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

# Load job descriptions and extracted CV details from JSON
with open('extracted_details.json', 'r') as json_file:
    cv_details = json.load(json_file)

# Sample job descriptions (replace with your own data)
job_descriptions = [
    "We are looking for a software engineer with expertise in Python and machine learning.",
    "Seeking a project manager with a background in construction management.",
    "Hiring a data scientist with experience in deep learning and natural language processing.",
]

# Initialize a dictionary to store top 5 CVs for each job description
top_5_cvs = {}

# Tokenize and embed job descriptions
job_desc_embeddings = [model(**tokenizer(job_desc, return_tensors='pt')).last_hidden_state.mean(dim=1) for job_desc in job_descriptions]

# Tokenize and embed CV details
for cv in cv_details:
    cv_text = f"{cv['Category']} {', '.join(cv['Skills'])} {', '.join(cv['Education'])}"
    cv_embedding = model(**tokenizer(cv_text, return_tensors='pt')).last_hidden_state.mean(dim=1)

    # Calculate cosine similarity between job descriptions and CVs
    cv_embedding=cv_embedding.detach().numpy()

    print(type(cv_embedding))
    print(type(job_desc_embeddings))
    similarities = cosine_similarity(job_desc_embeddings, cv_embedding)

    # Find the top 5 CVs for each job description
    for i, job_desc in enumerate(job_descriptions):
        if job_desc not in top_5_cvs:
            top_5_cvs[job_desc] = []

        top_5_indices = similarities[i].argsort()[-5:][::-1]
        top_5_cvs[job_desc].extend([(cv['PDFFilename'], similarities[i][idx]) for idx in top_5_indices])

# Print the top 5 CVs for each job description
for job_desc, cvs in top_5_cvs.items():
    print(f"Top 5 CVs for '{job_desc}':")
    for cv in cvs:
        print(f"CV: {cv[0]}, Similarity Score: {cv[1]}")
    print()


<class 'numpy.ndarray'>
<class 'list'>


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.